In [2]:
#Clean movies_metadata.csv

import pandas as pd
import ast

# Load movie dataset
df = pd.read_csv(r'/Users/bhoomikaharkhani/Downloads/movie_dataset/movies_metadata.csv', low_memory = False)

# Keep relevant columns
df = df[['id', 'title', 'genres', 'release_date', 'runtime', 'vote_average', 'vote_count']]

# Remove non-numeric IDs
df = df[df['id'].apply(lambda x: str(x).isdigit())]
df['movie_id'] = df['id'].astype(int)

# Extract first genre from JSON-like string
def extract_genre(genre_str):
    try:
        genres = ast.literal_eval(genre_str)
        return genres[0]['name'] if genres else none
    except:
        return None

df['genre'] = df['genres'].apply(extract_genre)

# Convert release date to datetime
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Rename columns and select final ones
df = df.rename(
    columns={
        'title':'movie_name',
        'runtime':'duration_mins',
        'vote_average':'average_rating',
        'vote_count':'rating_count'
    })

df = df[['movie_id', 'movie_name', 'genre', 'release_date', 'duration_mins', 'average_rating', 'rating_count']]

# Save cleaned file
df.to_csv('dim_movie_clean.csv', index=False)


dim_movie_clean.csv saved.


In [1]:
#Clean ratings_small.csv

import pandas as pd
from datetime import datetime

# Load ratings dataset
ratings = pd.read_csv(r'/Users/bhoomikaharkhani/Downloads/movie_dataset/ratings_small.csv', low_memory = False)

# Convert timestamp to date
ratings['date'] = pd.to_datetime(ratings['timestamp'], unit='s')

# Add rating_id (auto increment)
ratings.insert(0, 'rating_id', range(1, len(ratings) + 1))

# Select required columns
fact_rating = ratings[['rating_id', 'movieId', 'userId', 'date', 'rating']]
fact_rating = fact_rating.rename(columns={
    'movieId': 'movie_id',
    'userId': 'user_id'
})

# Save
fact_rating.to_csv('fact_rating_clean.csv', index=False)


✅ fact_rating_clean.csv saved.


In [2]:
#Create dim_user.csv

import numpy as np

#Get unique user_ids from ratings
user_ids = ratings['userId'].drop_duplicates().reset_index(drop=True)

# Simulate additional user details
dim_user = pd.DataFrame()
dim_user['user_id'] = user_ids
dim_user['user_name'] = user_ids.apply(lambda x: f'User_{x}')
dim_user['gender'] = np.random.choice(['Male', 'Female'], size = len(user_ids))
dim_user['country'] = np.random.choice(['USA','India','UK','Germany'], size=len(user_ids))

dim_user.to_csv('dim_user.csv', index=False)
print("✅ dim_user.csv saved.")


✅ dim_user.csv saved.


In [3]:
# Extract unique dates from ratings
unique_dates = pd.to_datetime(fact_rating['date'].unique())
dim_date = pd.DataFrame()
dim_date['date'] = unique_dates
dim_date['year'] = dim_date['date'].dt.year
dim_date['month'] = dim_date['date'].dt.month
dim_date['month_name'] = dim_date['date'].dt.strftime('%B')
dim_date['quarter'] = dim_date['date'].dt.quarter.apply(lambda q: f'Q{q}')
dim_date['week_number'] = dim_date['date'].dt.isocalendar().week

dim_date.to_csv('dim_date.csv', index=False)
print("✅ dim_date.csv saved.")

✅ dim_date.csv saved.


In [2]:
import pandas as pd

df = pd.read_csv("dim_date.csv")
df['date'] = pd.to_datetime(df['date']).dt.date  # Adjust column name
df = df.drop_duplicates(subset='date')
df.to_csv("dim_date_clean.csv", index=False)
print("✅ dim_date.csv saved.")

✅ dim_date.csv saved.
